
# AutoGluon 분류 파이프라인 (Stratified K-Fold, 셀별 실행 · 함수형태 X)

**목표**: `train.csv`(타깃: `target`)로 학습 → `test.csv` 분류 결과 생성  
**요구사항 반영**
- Stratified K-Fold (OOF & 폴드 성능)
- **각 기능별 셀**로 구현(함수 정의 없이 바로 실행)
- 데이터 전처리:
  - **G1 그룹**: `["X_05","X_09","X_20","X_22","X_25","X_51"]` → 표준화(평균0, 분산1) → **PCA(설명분산 0.95)** → `PC_G1_1, PC_G1_2, ...`만 남기고 G1 원본 6개 변수 **삭제**
  - **PAIR_GROUPS(쌍 그룹)**: 각 쌍에서 대표 1개만 남김. **대표 선택 기준**: 결측률↓, mutual information(↑), ANOVA-F(↑)
- AutoGluon으로 학습/검증/추론


In [45]:

# === 1. Imports & Config ===
import os
from pathlib import Path
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif, f_classif

try:
    from autogluon.tabular import TabularDataset, TabularPredictor
    AUTOGluon_OK = True
except Exception as e:
    AUTOGluon_OK = False
    print('[WARN] AutoGluon import 실패:', e)

# 경로/설정
DATA_DIR = Path('.')
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH  = DATA_DIR / 'test.csv'
TARGET_COL = 'target'
ID_COL = None  # 예: 'id' (없으면 None)

N_FOLDS = 5
RANDOM_STATE = 42
PRESETS = 'medium_quality'
TIME_LIMIT = 1800  # 초(폴드당)

SAVE_ROOT = Path('./ag_cv_models')
OOF_PATH  = Path('./oof_predictions.csv')
FULL_DIR  = Path('./ag_full_model')
SUB_PATH  = Path('./submission.csv')

# 그룹 정의
# 상관 계수 0.95 이상인 쌍
# 파생 변수 만들 예정
# 파생 변수 만들고 컬럼을 삭제 안하는 게 더 성능이 좋게 나옴.(왜지?)
G1 = ["X_05","X_09","X_20","X_22","X_25","X_51"]
VAR_RATIO = 0.95  # PCA 설명분산 임계

# 일단 여기선 별로 사용이 안 됨.
# 쌍 변수 그룹 (필요시 더 추가)
# 그 다음으로 상관 계수 높은 쌍
# 얘네들은 한 쌍에서 한 feature는 죽이기
PAIR_GROUPS = [
    ["X_04","X_39"],
    ["X_06","X_45"],
    ####################
    ["X_10","X_17"],
    ["X_07","X_33"],
    ["X_12","X_21"],
    ["X_26","X_30"],
    ["X_38","X_47"],
    
    # ["X_05","X_25"], ...  # 예시로 더 넣을 수 있음
]

# 재현성
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

print('[INFO] Config loaded.')

[INFO] Config loaded.


In [46]:

# === 2. Load train/test, 타깃 점검 ===
assert TRAIN_PATH.exists(), f'train.csv가 {TRAIN_PATH} 에 없습니다.'
train = pd.read_csv(TRAIN_PATH)
print('[INFO] train.shape =', train.shape)
assert TARGET_COL in train.columns, f'{TARGET_COL} 컬럼이 train에 없습니다.'

if TEST_PATH.exists():
    test = pd.read_csv(TEST_PATH)
    print('[INFO] test.shape  =', test.shape)
else:
    test = None
    print('[WARN] test.csv 가 없어 추론 셀은 스킵될 수 있습니다.')

# 타깃 클린업(문자열 통일 + 결측 제거)
y = train[TARGET_COL].astype(str)
na_mask = y.isna() | (y.str.lower()=='nan') | (y=='None') | (y=='')
if na_mask.any():
    print(f'[WARN] 타깃 결측/유사결측 {na_mask.sum()}건 발견 → 해당 행 제거')
    train = train.loc[~na_mask].reset_index(drop=True)
    y = train[TARGET_COL].astype(str)

ALL_CLASSES = sorted(y.unique())
print('[INFO] 클래스 수:', len(ALL_CLASSES))
print('[INFO] 예시:', ALL_CLASSES[:20])

[INFO] train.shape = (21693, 54)
[INFO] test.shape  = (15004, 53)
[INFO] 클래스 수: 21
[INFO] 예시: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '3', '4', '5', '6', '7', '8']


In [47]:

# === 3. PAIR_GROUPS 전역 대표 변수 선택 (train 기준) ===
# - 결측률 낮을수록 좋음
# - mutual information 높을수록 좋음
# - ANOVA-F 높을수록 좋음
# 점수 = (-결측률) 정규화 + (MI) 정규화 + (F) 정규화  (각 1/3 가중)

# 고상관 쌍(PAIR_GROUPS)의 각 쌍에서 대표 변수 1개만 남기기 위해, 두 변수의 “품질+유용성”을 점수로 계산해 비교하는 로직
# -> 39, 45 삭제

rep_map = {}         # {('X_04','X_39'): 'X_04', ...}
to_drop_pairs = []   # 대표가 아닌 변수 목록

# 스코어 계산을 위한 임시 데이터(결측 채움: 수치 median) 구성
num_cols = train.drop(columns=[TARGET_COL]).select_dtypes(include=[np.number]).columns.tolist()
tmp = train.copy()

if len(num_cols) > 0:
    imputer_num = SimpleImputer(strategy='median')
    tmp[num_cols] = imputer_num.fit_transform(tmp[num_cols])

# 각 쌍에 대해 점수 산출
for pair in PAIR_GROUPS:
    pair = [c for c in pair if c in train.columns and c != TARGET_COL]
    if len(pair) != 2:
        print(f'[WARN] 쌍 {pair} 중 유효 컬럼이 2개가 아닙니다. 스킵.')
        continue

    cols_ok = []
    scores = []
    for col in pair:
        # 결측률 (원본 기준)
        miss_rate = train[col].isna().mean()

        # MI/F는 수치형에서만 바로 계산. 수치가 아닐 경우 임시 변환(범주→순번 인코딩)
        s = tmp[col]
        if not np.issubdtype(s.dtype, np.number):
            # 간단 라벨 인코딩
            s = s.astype('category').cat.codes

        X_ = s.values.reshape(-1, 1)
        y_ = train[TARGET_COL].astype(str).values

        # MI (discrete target)
        try:
            mi = mutual_info_classif(X_, y_, discrete_features=True, random_state=RANDOM_STATE)[0]
        except Exception:
            mi = 0.0

        # ANOVA-F
        try:
            f, _ = f_classif(X_, pd.factorize(y_)[0])
            f = float(f[0])
        except Exception:
            f = 0.0

        cols_ok.append(col)
        scores.append({'col': col, 'miss': miss_rate, 'mi': mi, 'f': f})

    if len(scores) != 2:
        print(f'[WARN] {pair} 점수 계산 실패(스킵)')
        continue

    # 정규화
    miss_vals = np.array([s['miss'] for s in scores])
    mi_vals   = np.array([s['mi']   for s in scores])
    f_vals    = np.array([s['f']    for s in scores])

    def norm(v):
        v = v.astype(float)
        if np.allclose(v.max(), v.min()):
            return np.zeros_like(v)
        return (v - v.min()) / (v.max() - v.min())

    miss_n = norm(1 - miss_vals)  # 결측률 낮을수록↑ → (1 - miss)
    mi_n   = norm(mi_vals)
    f_n    = norm(f_vals)

    final  = (miss_n + mi_n + f_n) / 3.0
    best_idx = int(np.argmax(final))
    rep = scores[best_idx]['col']
    rep_map[tuple(pair)] = rep

    drop_cols = [c for c in pair if c != rep]
    to_drop_pairs.extend(drop_cols)

print('[INFO] 대표 선택 결과:', rep_map)
print('[INFO] 삭제 대상(대표 아닌 변수):', to_drop_pairs)

c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and

[INFO] 대표 선택 결과: {('X_04', 'X_39'): 'X_04', ('X_06', 'X_45'): 'X_06', ('X_10', 'X_17'): 'X_17', ('X_07', 'X_33'): 'X_07', ('X_12', 'X_21'): 'X_12', ('X_26', 'X_30'): 'X_26', ('X_38', 'X_47'): 'X_38'}
[INFO] 삭제 대상(대표 아닌 변수): ['X_39', 'X_45', 'X_10', 'X_33', 'X_21', 'X_30', 'X_47']


c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\SSAFY\Downloads\Autogluon\venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and

In [48]:

# === 4. 대표 아닌 변수 삭제(전역) ===
train_pair = train.drop(columns=[c for c in to_drop_pairs if c in train.columns], errors='ignore').copy()
if test is not None:
    test_pair = test.drop(columns=[c for c in to_drop_pairs if c in test.columns], errors='ignore').copy()
else:
    test_pair = None

print('[INFO] train_pair.shape =', train_pair.shape)
if test_pair is not None:
    print('[INFO] test_pair.shape  =', test_pair.shape)

[INFO] train_pair.shape = (21693, 47)
[INFO] test_pair.shape  = (15004, 46)


In [49]:
train_pair.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18',
       'X_19', 'X_20', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28',
       'X_29', 'X_31', 'X_32', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38', 'X_40',
       'X_41', 'X_42', 'X_43', 'X_44', 'X_46', 'X_48', 'X_49', 'X_50', 'X_51',
       'X_52', 'target'],
      dtype='object')

In [50]:

# === 파생 피처 추가 (함수 없이 바로 적용) ===
_base_cols = [c for c in train_pair.columns if c != TARGET_COL]
_num_cols  = train_pair[_base_cols].select_dtypes(include=[float, int]).columns

# 1) 행 단위 요약
train_pair['row_na_cnt'] = train_pair[_base_cols].isna().sum(axis=1)
if test_pair is not None:
    test_pair['row_na_cnt'] = test_pair[_base_cols].isna().sum(axis=1)

train_pair['row_num_mean'] = train_pair[_num_cols].mean(axis=1)
train_pair['row_num_std']  = train_pair[_num_cols].std(axis=1)
if test_pair is not None:
    test_pair['row_num_mean'] = test_pair[_num_cols].mean(axis=1)
    test_pair['row_num_std']  = test_pair[_num_cols].std(axis=1)

if len(_num_cols) > 0:
    _Q1  = train_pair[_num_cols].quantile(0.25)
    _Q3  = train_pair[_num_cols].quantile(0.75)
    _IQR = (_Q3 - _Q1).replace(0, 1e-12)

    _below_tr = (train_pair[_num_cols] < (_Q1 - 1.5 * _IQR))
    _above_tr = (train_pair[_num_cols] > (_Q3 + 1.5 * _IQR))
    train_pair['row_outlier_cnt'] = (_below_tr | _above_tr).sum(axis=1)

    if test_pair is not None:
        _below_te = (test_pair[_num_cols] < (_Q1 - 1.5 * _IQR))
        _above_te = (test_pair[_num_cols] > (_Q3 + 1.5 * _IQR))
        test_pair['row_outlier_cnt'] = (_below_te | _above_te).sum(axis=1)

# 2) G1 그룹 통계
_g1_cols = [c for c in G1 if c in train_pair.columns]
if len(_g1_cols) > 0:
    train_pair['G1_mean']  = train_pair[_g1_cols].mean(axis=1)
    train_pair['G1_std']   = train_pair[_g1_cols].std(axis=1)
    train_pair['G1_min']   = train_pair[_g1_cols].min(axis=1)
    train_pair['G1_max']   = train_pair[_g1_cols].max(axis=1)
    train_pair['G1_range'] = train_pair['G1_max'] - train_pair['G1_min']

    if test_pair is not None:
        test_pair['G1_mean']  = test_pair[_g1_cols].mean(axis=1)
        test_pair['G1_std']   = test_pair[_g1_cols].std(axis=1)
        test_pair['G1_min']   = test_pair[_g1_cols].min(axis=1)
        test_pair['G1_max']   = test_pair[_g1_cols].max(axis=1)
        test_pair['G1_range'] = test_pair['G1_max'] - test_pair['G1_min']


# 3) 비선형 파생 (예: X_05, X_09)
if 'X_05' in train_pair.columns and 'X_09' in train_pair.columns:
    train_pair['X05_div_X09'] = train_pair['X_05'] / (train_pair['X_09'].replace(0, 1e-12) + 1e-12)
    train_pair['X05_mul_X09'] = train_pair['X_05'] * train_pair['X_09']
    if test_pair is not None:
        test_pair['X05_div_X09'] = test_pair['X_05'] / (test_pair['X_09'].replace(0, 1e-12) + 1e-12)
        test_pair['X05_mul_X09'] = test_pair['X_05'] * test_pair['X_09']

# if 'X_05' in train_pair.columns:
#     train_pair['X05_sq'] = train_pair['X_05'] ** 2
#     if test_pair is not None:
#         test_pair['X05_sq'] = test_pair['X_05'] ** 2

# if 'X_09' in train_pair.columns:
#     train_pair['X09_sq'] = train_pair['X_09'] ** 2
#     if test_pair is not None:
#         test_pair['X09_sq'] = test_pair['X_09'] ** 2


if 'X_20' in train_pair.columns and 'X_22' in train_pair.columns:
    train_pair['X20_div_X22'] = train_pair['X_20'] / (train_pair['X_22'].replace(0, 1e-12) + 1e-12)
    train_pair['X20_mul_X22'] = train_pair['X_20'] * train_pair['X_22']
    if test_pair is not None:
        test_pair['X20_div_X22'] = test_pair['X_20'] / (test_pair['X_22'].replace(0, 1e-12) + 1e-12)
        test_pair['X20_mul_X22'] = test_pair['X_20'] * test_pair['X_22']

# if 'X_20' in train_pair.columns:
#     train_pair['X20_sq'] = train_pair['X_20'] ** 2
#     if test_pair is not None:
#         test_pair['X20_sq'] = test_pair['X_20'] ** 2

# if 'X_22' in train_pair.columns:
#     train_pair['X22_sq'] = train_pair['X_22'] ** 2
#     if test_pair is not None:
#         test_pair['X22_sq'] = test_pair['X_22'] ** 2



if 'X_25' in train_pair.columns and 'X_51' in train_pair.columns:
    train_pair['X25_div_X51'] = train_pair['X_25'] / (train_pair['X_51'].replace(0, 1e-12) + 1e-12)
    train_pair['X25_mul_X51'] = train_pair['X_25'] * train_pair['X_51']
    if test_pair is not None:
        test_pair['X25_div_X51'] = test_pair['X_25'] / (test_pair['X_51'].replace(0, 1e-12) + 1e-12)
        test_pair['X25_mul_X51'] = test_pair['X_25'] * test_pair['X_51']

# if 'X_25' in train_pair.columns:
#     train_pair['X25_sq'] = train_pair['X_25'] ** 2
#     if test_pair is not None:
#         test_pair['X25_sq'] = test_pair['X_25'] ** 2

# if 'X_51' in train_pair.columns:
#     train_pair['X51_sq'] = train_pair['X_51'] ** 2
#     if test_pair is not None:
#         test_pair['X51_sq'] = test_pair['X_51'] ** 2


print('[INFO] 파생 피처 추가 완료. train_pair:', train_pair.shape, '| test_pair:', (None if test_pair is None else test_pair.shape))


[INFO] 파생 피처 추가 완료. train_pair: (21693, 62) | test_pair: (15004, 61)


In [51]:

# === 5. Stratified K-Fold CV (fold별로 G1 표준화+PCA 적용 후 AutoGluon 학습) ===
if not AUTOGluon_OK:
    raise ImportError('AutoGluon 불러오기 실패. 설치 필요: pip install autogluon.tabular')

SAVE_ROOT.mkdir(parents=True, exist_ok=True)

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
fold_indices = list(skf.split(train_pair.drop(columns=[TARGET_COL]), train_pair[TARGET_COL].astype(str)))

oof_pred = pd.Series(index=np.arange(len(train_pair)), dtype=object)
fold_scores = []

for fold, (trn_idx, val_idx) in enumerate(fold_indices, start=1):
    print(f'\n===== [Fold {fold}/{N_FOLDS}] =====')
    trn_df = train_pair.iloc[trn_idx].reset_index(drop=True).copy()
    val_df = train_pair.iloc[val_idx].reset_index(drop=True).copy()

    # --- G1 처리: 표준화 → PCA(0.95) → PC_G1_* 생성, G1 원본 삭제 ---
    g1_cols = [c for c in G1 if c in trn_df.columns and c != TARGET_COL]
    if len(g1_cols) > 0:
        # 결측 채우기(수치 median) → 스케일러/ PCA 학습은 trn 기준
        imputer_g1 = SimpleImputer(strategy='median')
        scaler_g1  = StandardScaler()
        pca_g1     = PCA(n_components=VAR_RATIO, svd_solver='full')

        trn_g1 = imputer_g1.fit_transform(trn_df[g1_cols])
        trn_g1 = scaler_g1.fit_transform(trn_g1)
        trn_g1_pc = pca_g1.fit_transform(trn_g1)

        # 검증 변환
        val_g1 = imputer_g1.transform(val_df[g1_cols])
        val_g1 = scaler_g1.transform(val_g1)
        val_g1_pc = pca_g1.transform(val_g1)

        # PC 열명 만들기
        n_pc = trn_g1_pc.shape[1]
        pc_cols = [f'PC_G1_{i+1}' for i in range(n_pc)]

        # 데이터프레임에 붙이고 원본 G1 삭제
        trn_pc_df = pd.DataFrame(trn_g1_pc, columns=pc_cols)
        val_pc_df = pd.DataFrame(val_g1_pc, columns=pc_cols)

        trn_df = pd.concat([trn_df.drop(columns=g1_cols), trn_pc_df], axis=1)
        val_df = pd.concat([val_df.drop(columns=g1_cols), val_pc_df], axis=1)
    else:
        print('[INFO] 이번 fold에서 G1 컬럼이 존재하지 않아 PCA 스킵')


    # AutoGluon 학습
    fold_dir = SAVE_ROOT / f'fold_{fold}'
    if fold_dir.exists():
        print(f'[INFO] {fold_dir} 재사용/덮어쓰기')
    fold_dir.mkdir(parents=True, exist_ok=True)

    ag_trn = TabularDataset(trn_df)
    ag_val = TabularDataset(val_df)

    predictor = TabularPredictor(label=TARGET_COL, path=str(fold_dir), eval_metric='f1_macro')
    predictor.fit(train_data=ag_trn, tuning_data=ag_val, presets=PRESETS, time_limit=TIME_LIMIT, verbosity=2)
    # predictor.fit(train_data=ag_trn, presets=PRESETS, time_limit=TIME_LIMIT, verbosity=2)

    # 검증 예측
    y_true = val_df[TARGET_COL].astype(str).reset_index(drop=True)
    y_pred = predictor.predict(ag_val).astype(str).reset_index(drop=True)

    score = f1_score(y_true, y_pred, average='macro', labels=ALL_CLASSES, zero_division=0)
    print(f'[Fold {fold}] macro-F1: {score:.4f}')
    fold_scores.append(score)

    oof_pred.iloc[val_idx] = y_pred.values

# OOF 성능
y_all = train_pair[TARGET_COL].astype(str).reset_index(drop=True)
assert oof_pred.isna().sum() == 0, 'OOF 예측에 결측이 있습니다.'

oof_score = f1_score(y_all, oof_pred.astype(str), average='macro', labels=ALL_CLASSES, zero_division=0)
print('\n===== CV 결과 =====')
print('Fold scores:', np.round(fold_scores, 4).tolist())
print(f'OOF macro-F1: {oof_score:.4f}')

oof_df = pd.DataFrame({'y_true': y_all, 'y_pred': oof_pred.astype(str)})
oof_df.to_csv(OOF_PATH, index=False, encoding='utf-8')
print('[INFO] OOF 저장:', OOF_PATH.resolve())

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          28
Memory Avail:       16.36 GB / 31.69 GB (51.6%)
Disk Space Avail:   819.80 GB / 953.01 GB (86.0%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'



===== [Fold 1/5] =====
[INFO] ag_cv_models\fold_1 재사용/덮어쓰기


Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "c:\Users\SSAFY\Downloads\Autogluon\ag_cv_models\fold_1"
Train Data Rows:    17354
Train Data Columns: 56
Tuning Data Rows:    4339
Tuning Data Columns: 56
Label Column:       target
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 21) unique label values:  [np.int64(0), np.int64(20), np.int64(1), np.int64(19), np.int64(15), np.int64(8), np.int64(12), np.int64(4), np.int64(5), np.int64(11)]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 21
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:              

[Fold 1] macro-F1: 0.8716

===== [Fold 2/5] =====
[INFO] ag_cv_models\fold_2 재사용/덮어쓰기


Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "c:\Users\SSAFY\Downloads\Autogluon\ag_cv_models\fold_2"
Train Data Rows:    17354
Train Data Columns: 56
Tuning Data Rows:    4339
Tuning Data Columns: 56
Label Column:       target
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 21) unique label values:  [np.int64(20), np.int64(1), np.int64(15), np.int64(0), np.int64(8), np.int64(16), np.int64(14), np.int64(18), np.int64(3), np.int64(5)]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 21
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:              

...

[Fold 3] macro-F1: 0.8776

===== [Fold 4/5] =====
[INFO] ag_cv_models\fold_4 재사용/덮어쓰기


Train Data Class Count: 21
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    17378.43 MB
	Train Data (Original)  Memory Usage: 10.51 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 1): ['row_na_cnt']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not 

[Fold 4] macro-F1: 0.8697

===== [Fold 5/5] =====
[INFO] ag_cv_models\fold_5 재사용/덮어쓰기


Train Data Class Count: 21
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    16952.99 MB
	Train Data (Original)  Memory Usage: 10.51 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 1): ['row_na_cnt']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not 

[Fold 5] macro-F1: 0.8709

===== CV 결과 =====
Fold scores: [0.8716, 0.8694, 0.8776, 0.8697, 0.8709]
OOF macro-F1: 0.8720
[INFO] OOF 저장: C:\Users\SSAFY\Downloads\Autogluon\oof_predictions.csv


In [52]:

# === 6. Full-train + Test 예측 (전역 대표/드롭 적용 기준, G1은 full-train 기준으로 PCA) ===
if test is None:
    print('[WARN] test.csv 없음 → 스킵')
else:
    # 전역 pair-drop 반영된 사본 사용
    trn_full = train_pair.copy()
    tst_full = test_pair.copy()

    # G1 처리: 결측→표준화→PCA(0.95) → PC_G1_* 남기고 원본 삭제 (full-train 기준으로 적합)
    g1_cols_full = [c for c in G1 if c in trn_full.columns and c != TARGET_COL]
    if len(g1_cols_full) > 0:
        imputer_g1_full = SimpleImputer(strategy='median')
        scaler_g1_full  = StandardScaler()
        pca_g1_full     = PCA(n_components=VAR_RATIO, svd_solver='full')

        trn_g1f = imputer_g1_full.fit_transform(trn_full[g1_cols_full])
        trn_g1f = scaler_g1_full.fit_transform(trn_g1f)
        trn_g1f_pc = pca_g1_full.fit_transform(trn_g1f)

        tst_g1f = imputer_g1_full.transform(tst_full[g1_cols_full])
        tst_g1f = scaler_g1_full.transform(tst_g1f)
        tst_g1f_pc = pca_g1_full.transform(tst_g1f)

        n_pc_full = trn_g1f_pc.shape[1]
        pc_cols_full = [f'PC_G1_{i+1}' for i in range(n_pc_full)]

        trn_pc_full = pd.DataFrame(trn_g1f_pc, columns=pc_cols_full)
        tst_pc_full = pd.DataFrame(tst_g1f_pc, columns=pc_cols_full)

        trn_full = pd.concat([trn_full.drop(columns=g1_cols_full), trn_pc_full], axis=1)
        tst_full = pd.concat([tst_full.drop(columns=g1_cols_full), tst_pc_full], axis=1)
    else:
        print('[INFO] full-train 기준 G1 컬럼 없음 → PCA 스킵')


    # AutoGluon 전체 학습
    FULL_DIR.mkdir(parents=True, exist_ok=True)
    ag_full = TabularDataset(trn_full)
    full_predictor = TabularPredictor(label=TARGET_COL, path=str(FULL_DIR), eval_metric='f1_macro')
    full_predictor.fit(train_data=ag_full, presets=PRESETS, time_limit=TIME_LIMIT, verbosity=2)

    # Test 예측 & 제출 저장
    ag_test = TabularDataset(tst_full)
    test_pred = full_predictor.predict(ag_test).astype(str)

    if ID_COL is not None and ID_COL in test.columns:
        sub = pd.DataFrame({ID_COL: test[ID_COL], TARGET_COL: test_pred})
    else:
        sub = pd.DataFrame({'row_id': np.arange(len(test_pred)), TARGET_COL: test_pred})

    sub.to_csv(SUB_PATH, index=False, encoding='utf-8')
    print('[INFO] submission 저장:', SUB_PATH.resolve())

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          28
Memory Avail:       16.12 GB / 31.69 GB (50.9%)
Disk Space Avail:   819.78 GB / 953.01 GB (86.0%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "c:\Users\SSAFY\Downloads\Autogluon\ag_full_model"
Train Data Rows:    21693
Train Data Columns: 56
Label Column:       target
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 21) unique label values:  [np.int64(0), np.int64(20), np.int64(1), np.int64(19), np.int64(15), np.int64(8), np.int64(16), np.int64(12), np.int64(14), np.int64(18)]
	If 'multiclass' is not the correct prob

[1000]	valid_set's multi_logloss: 0.652526	valid_set's f1_macro: 0.785492


	0.789	 = Validation score   (f1_macro)
	17.71s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 1762.65s of the 1762.65s of remaining time.
	Fitting with cpus=20, gpus=0, mem=0.2/15.9 GB
	0.8012	 = Validation score   (f1_macro)
	9.59s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestGini ... Training model for up to 1752.43s of the 1752.43s of remaining time.
	Fitting with cpus=28, gpus=0, mem=0.3/16.0 GB
	0.762	 = Validation score   (f1_macro)
	4.8s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 1747.29s of the 1747.29s of remaining time.
	Fitting with cpus=28, gpus=0, mem=0.3/15.4 GB
	0.7664	 = Validation score   (f1_macro)
	10.77s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 1736.21s of the 1736.21s of remaining time.
	Fitting with cpus=20, gpus=0
	0.7978	 = Validation score   

[INFO] submission 저장: C:\Users\SSAFY\Downloads\Autogluon\submission.csv



## 전처리 메모 (요약)
- **PAIR_GROUPS 대표 선택**: `(-결측률) 정규화 + MI 정규화 + F 정규화` 평균으로 대표 1개 선택(전역 고정). 대표 외 변수는 **삭제**.
- **G1 그룹**: 수치 결측 median 대체 → 표준화(`StandardScaler`) → `PCA(n_components=0.95)` → `PC_G1_i`만 유지, G1 원본 **삭제**.
- **K-Fold 내 누수 방지**: G1의 `imputer/scaler/PCA`는 **각 폴드의 train 부분으로만 적합** 후 val에 적용. (전역 pair-drop은 기준 통일을 위해 train 전체에서 1회 산정)
- AutoGluon은 나머지 결측/범주형 처리를 자체적으로 수행.
